## 2. Preprocessing: automated spot detection

In this step, spots are automatically detected and tracked for one of the experiments.

In [1]:
import os, re, glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from colicycle.MoMAobj import Momaobj
import colicycle.time_mat_operations as tmo
import colicycle.momaprocess as mp
import colicycle.spot_detection as sp

In [2]:
folder_prefix = '../PreProcessed/'

In [3]:
data_folder = folder_prefix+'20170327_GW339_temp/'
tosave_folder = data_folder+'step2/'

In [4]:
#load lane information 
mom = Momaobj(data_folder=data_folder,col_nb=2, no_raw=True)
#recover experiment name
current_name = data_folder.split('/')[np.argwhere([re.findall('^[0-9]{8,9}', x) for x in data_folder.split('/')])[0][0]]

In [7]:
for i in range(len(mom.pos_list)):
    print(i)
    mom.pos = mom.pos_list[i]
    mom.gl = mom.gl_list[i]
    
    filename = data_folder+'/step1/step1_time_mat'+'_pos'+mom.pos+'_GL'+mom.gl+'.pkl'
    time_mat_pd = pd.read_pickle(filename)
    time_mat_pd.loc[time_mat_pd.born ==-1,'born'] = 0
    
    #find spots
    time_mat_pd = tmo.find_spots(time_mat_pd=time_mat_pd,sigmaXY=1.5,mom=mom)
    
    #keep only spots with good cross-correlation score
    time_mat_pd['spots'] = time_mat_pd.spots.apply(lambda x: x[x[:,9]>0.6,:] if len(x)>0 else [])
    
    #track spots and define oriC splitting
    time_mat_pd = tmo.track_spots(time_mat_pd)
    
    #calculate Li_fit
    #time_mat_pd['Li_fit'] = time_mat_pd.apply(lambda row: tmo.length_fit_at_T(row, 'Ti'),axis = 1)
    
    #save file with spot information
    filename = data_folder+'/step2/automatic_time_mat'+'_pos'+mom.pos+'_GL'+mom.gl+'.pkl'
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    time_mat_pd.to_pickle(filename)

Frame 36: 4 trajectories present.
